### This notebook presents an execution tree enhanced with provenance.
### In this view, the nodes that are not function calls are removed 

In [1]:
# Imports
import sqlite3

from graphviz import Graph

from pyalgdb.validity import Validity
from pyalgdb.node import Node
from pyalgdb.execution_tree import ExecutionTree
from pyalgdb.execution_tree_creator import ExecTreeCreator
from pyalgdb.top_down import TopDown
from pyalgdb.heaviest_first import HeaviestFirst
from pyalgdb.visualization import Visualization
from pyalgdb.provenance_enhancement import ProvenanceEnhancement 
from pyalgdb.single_stepping import SingleStepping
from pyalgdb.divide_and_query import DivideAndQuery

In [2]:
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/ws/py-scripts-examples/min-max/.noworkflow/db.sqlite'

In [3]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()

In [4]:
creator = ExecTreeCreator(CURSOR)
exec_tree = creator.create_exec_tree()

In [5]:
prov = ProvenanceEnhancement(exec_tree, CURSOR)

In [6]:
dependencies = prov.prov_tools.get_dependencies()
prov.dependencies = dependencies

In [7]:
# Perform the provenance enhancement, having as "start set" all nodes that are function calls
for d in dependencies:
    if d.influencer.code_component_type == 'call':
        prov.enhance(d.influencer)
    if d.dependent.code_component_type == 'call':
        prov.enhance(d.dependent)

In [8]:
class CustomVisualization(Visualization):

    PROVENANCE_COLOR = 'dodgerblue'
    PROVENANCE_NODE_COLOR = 'lightblue'
    
    def view_exec_tree_prov(self, graph_name, dependencies:list):
        file_name = "{}.gv".format(graph_name)
        self.graph = Graph(graph_name, filename=file_name)
        self.graph.attr('node', shape='box')
        self.graph.attr('graph', ordering='out')
        root_node = self.exec_tree.root_node
        self.graph.node(str(root_node.ev_id), root_node.name, fillcolor=self.INVALID_COLOR, style='filled')
        self.navigate(root_node)
        eval_node = self.exec_tree.node_under_evaluation
        if eval_node is not None:
            self.graph.node(str(eval_node.ev_id), str(eval_node.name), fillcolor=self.NODE_IN_EVALUATION, style='filled')
        for d in dependencies: # this loop draws the provenance links between nodes
            self.graph.edge(str(d.dependent.ev_id), str(d.influencer.ev_id), None, color=self.PROVENANCE_COLOR, dir='forward')
            self.graph.node(str(d.dependent.ev_id), None, fillcolor=self.PROVENANCE_NODE_COLOR, style='filled')
            self.graph.node(str(d.influencer.ev_id), None, fillcolor=self.PROVENANCE_NODE_COLOR, style='filled')
        self.graph.view()

    

In [9]:
vis = CustomVisualization(prov.exec_tree)

In [10]:
vis.view_exec_tree_prov('exec_tree_p1', prov.final_dependencies)

In [11]:
######################################################################################################################

### The following cells presents an execution tree enhanced with provenance.
### In this view, the wrong Evaluation is asked, and the provenance DAG is built based on the slice of the wrong evaluation. 

In [12]:
exec_tree = creator.create_exec_tree()

In [13]:
prov = ProvenanceEnhancement(exec_tree, CURSOR)

In [14]:
dependencies = prov.prov_tools.get_dependencies()
prov.dependencies = dependencies

In [15]:
wrong_data = prov.ask_wrong_data()

Which evaluation id is not correct? 31


In [16]:
wrong_data

In [17]:
prov.enhance(wrong_data)

In [18]:
vis = CustomVisualization(prov.exec_tree)

In [19]:
vis.view_exec_tree_prov('exec_tree_p2', prov.final_dependencies)